In [ ]:
import pandas as pd
import numpy as np

from google.colab import auth
from google.colab import drive
auth.authenticate_user()
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Colab Notebooks/프로젝트2/'

In [ ]:
orders=pd.read_csv(path+'orders.csv')

## **전체 사용자 평균 구매 주기**

In [ ]:
orders.groupby('user_id')['days_since_prior_order'].mean()

In [ ]:
#각 사용자의 평균 주문 간격
orders.groupby('user_id').agg({'days_since_prior_order':'mean'})

In [ ]:
# 전체 사용자 평균 구매 주기
total_prior_mean=orders.groupby('user_id').agg({'days_since_prior_order':'mean'})
total_prior_mean.mean()

In [ ]:
orders.loc[(orders['order_number']==50)]

In [ ]:
opp=pd.read_csv(path+'order_products__prior.csv')

In [ ]:
orders['eval_set'].value_counts()

## **재주문한 제품들과 사용자가 카트에 추가한 순서 간의 관계 확인**

In [ ]:
# orders와 opp 데이터셋을 병합
merged_data = pd.merge(orders, opp, on='order_id', how='inner')

In [ ]:
merged_data['add_to_cart_order'].value_counts()

In [ ]:
# 카트에 제품을 추가한 순서(add_to_cart_order)와 재주문 여부(reordered) 간의 관계 확인
cart_reorder_relation = merged_data.groupby('add_to_cart_order')['reordered'].mean()

In [ ]:
# cart_reorder_relation = 각 카트에 제품을 추가한 순서별로 평균 재주문 비율
cart_reorder_relation*100

**특정 순서에서의 재주문 확률을 확인**
1.   add_to_cart_order 컬럼을 기준으로 데이터를 그룹화:카트에 제품을 추가한 순서에 따라 데이터를 그룹화
2.   그룹화된 데이터에서 'reordered' 컬럼의 평균을 계산

*   'reordered'는 재주문 여부를 나타내는 컬럼, 값이 1이면 재주문을 의미하고 0이면 재주문하지 않은 것을 의미

*    평균을 계산하면 해당 순서에서의 평균 재주문 비율이 나옴

In [ ]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 설정
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))

# 'add_to_cart_order'에 따른 재주문 비율 시각화
sns.barplot(x=cart_reorder_relation.index, y=(cart_reorder_relation*100).values, color='skyblue')

# 그래프 설정
plt.title('Reorder Rate by Add to Cart Order')
plt.xlabel('Add to Cart Order')
plt.ylabel('Reorder Rate')
plt.show()

## **특정 시간대별 재주문 분석**




In [ ]:
#주문 수와 함께 해당 시간대의 재주문 비율을 시각적으로 비교
plt.figure(figsize=(12, 6))
sns.countplot(x='order_hour_of_day', hue='reordered', data=merged_data)
plt.title('Reorder Count by Order Hour of Day')
plt.xlabel('Order Hour of Day')
plt.ylabel('Count')
plt.legend(title='Reordered', loc='upper right')
plt.show()

In [ ]:
# 시간대별 주문 건수 계산
orders_by_hour = merged_data.groupby('order_hour_of_day').size()
orders_by_hour

In [ ]:
plt.figure(figsize=(12, 6))
orders_by_hour.plot(kind='bar', color='skyblue')
plt.title('시간대별 주문 건수')
plt.xlabel('주문 시간대')
plt.ylabel('주문 건수')
plt.show()

In [ ]:
# 사용자의 첫 번째 주문과 그 이후 주문 간의 시간 간격 계산
first_order_time = merged_data.groupby('user_id')['days_since_prior_order'].first()
first_order_time

In [ ]:
reordered_ratio2 = total.groupby('user_id')['reordered'].mean().reset_index()
reordered_ratio2*100

## **재구매 패턴 분석**
- "reordered" 컬럼을 사용하여 각 제품이 얼마나 자주 재주문되었는지 확인
- reorder_count 컬럼:해당 제품의 총 재주문 횟수

In [ ]:
product=pd.read_csv(path+'products.csv')

In [ ]:
# merged_data 와 product 데이터셋을 병합
merged_product = pd.merge(merged_data, product, on='product_id', how='left')
merged_product.head()

In [ ]:
# 각 제품별로 재주문 횟수 계산
reorder_counts = merged_product.groupby('product_id')['reordered'].sum().reset_index()
reorder_counts.columns = ['product_id', 'reorder_count']
reorder_counts

In [ ]:
user_retention = orders.groupby('user_id')['days_since_prior_order'].sum().reset_index()
user_retention.sort_values(by='days_since_prior_order',ascending=False).head(50)

### **주문을 자주 하는 고객들이 재주문을 더 많이 할 수 있다 (?)**
### **고객별 총 주문 횟수를 구해서 분석해볼까?**

In [ ]:
customer_order_count = orders.groupby('user_id')['order_number'].count().reset_index()
customer_order_count.columns = ['user_id', 'total_orders']
customer_order_count.sort_values(by='total_orders',ascending=False).head(50)

In [ ]:
reordered_ratio = opp.groupby('order_id')['reordered'].mean().reset_index()
reordered_ratio*100

In [ ]:
# 테이블에 유저아이디 추가
mg=pd.merge(customer_order_count,orders,on='user_id', how='left')
mg

In [ ]:
merge_table = pd.merge(reordered_ratio,mg, on='order_id', how='left')

merge_table.loc[:,['user_id','order_id','reordered']]
total= merge_table.loc[:,['user_id','order_id','reordered']]
total.sort_values(by='reordered',ascending=False).head(30)

In [ ]:
correlation = customer_order_count['total_orders'].corr(reordered_ratio['reordered'])
correlation


## 카테고리 재주문율 확인

In [ ]:
df1=pd.read_csv(path+'top 5% order and products.csv')
df2=pd.read_csv(path+'filtered_orders_high.csv')
df3=pd.read_csv(path+'aisles.csv')

In [ ]:
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df = pd.concat([merged_df, df3], ignore_index=True)

In [ ]:
reorder_rate_by_aisle = merged_df.groupby('aisle_id')['reordered'].mean().reset_index()
reorder_rate_by_aisle = reorder_rate_by_aisle.sort_values(by='reordered',ascending=False)
reorder_rate_by_aisle

In [ ]:
merged_df = pd.merge(reorder_rate_by_aisle, df3, left_on='aisle_id', right_on='aisle_id', how='left')
merged_df.head(10)

#### **재주문 비율: 사용자의 총 주문수에 대한 재주문된 품목의 주문수의 비율**
높은 재주문률 : 재주문율 >= 임계치(70-80%)

In [ ]:
orders=pd.read_csv(path+'top 5% order and products.csv')
user=pd.read_csv(path+'filtered_orders_high.csv')

In [ ]:
orders=pd.DataFrame(orders)
orders

In [ ]:
# order_id를 그룹화하고 각 product_id별 reordered된 수를 계산
reordered_counts = orders.groupby(['order_id', 'product_id'])['reordered'].sum().reset_index()
reordered_counts.head(20)

In [ ]:
# 계산된 reordered를 합한다. order_id별 reordered 갯수 알수 있음 = 이걸 재주문율 지표로 하기로 함
reordered_counts.groupby(['order_id'])['reordered'].sum()

In [ ]:
# order_id 별 재주문률 (★지표★)
reordered_ratio = orders.groupby('order_id')['reordered'].mean().reset_index()
reordered_ratio*100

In [ ]:
# 테이블에 유저아이디 추가
user= pd.DataFrame(user)
merge_table = pd.merge(orders, user, on='order_id', how='left') # 두 테이블을 조인

merge_table.loc[:,['user_id','order_id','product_id','reordered']]
total= merge_table.loc[:,['user_id','order_id','product_id','reordered']]
total

In [ ]:
#use_id 별 재주문률
reordered_ratio2 = total.groupby('user_id')['reordered'].mean().reset_index()
reordered_ratio2*100

In [ ]:
df=pd.read_csv(path+'top 5% order and products.csv')
df1=pd.read_csv(path+'filtered_orders_high.csv')

# 각 order_id별 상품의 개수 계산
order_product_count = df.groupby('order_id')['product_id'].count().reset_index()
order_product_count.columns = ['order_id', 'product_count']

# order_product_count DataFrame과 원본 df를 'order_id'를 기준으로 merge
df = pd.merge(df, order_product_count, on='order_id', how='left')

# 'product_count' col 값을 2로 나눈 값을 새로운 col로 추가
df['product_count_divided'] = df['product_count'] / 2

# 'product_count_divided' col을 기준으로 새로운 col 생성
df['procedure'] = np.where(df['add_to_cart_order'] <= df['product_count_divided'], 'front', 'back')

# 두 데이터셋을 'order_id'를 기준으로 merge
merged_df = pd.merge(df, df1, on='order_id', how='left')

# add_to_cart_order와 reordered 간의 관계 확인
order_reorder_relationship = df.groupby('add_to_cart_order')['reordered'].mean().reset_index()
order_reorder_relationship.columns = ['add_to_cart_order', 'mean_reordered']
order_reorder_relationship

# 각 카트에 제품을 추가한 순서별로 평균 재주문 비율 계산
average_reordered_by_order = df.groupby('add_to_cart_order')['reordered'].mean().reset_index()
average_reordered_by_order.columns = ['add_to_cart_order', 'average_reordered_ratio']
average_reordered_by_order

# 'add_to_cart_order'에 따른 재주문 비율 계산
average_reordered_by_order = df.groupby(['add_to_cart_order', 'procedure'])['reordered'].mean().reset_index()

# 시각화
plt.figure(figsize=(12, 6))
sns.lineplot(x='add_to_cart_order', y='reordered', data=average_reordered_by_order)
plt.title('Reorder Ratio by Add to Cart Order')
plt.xlabel('Add to Cart Order')
plt.ylabel('Reorder Ratio')

# x축 간격 조절
plt.xticks(np.arange(1, df['add_to_cart_order'].max(), step=df['add_to_cart_order'].max()//20))

plt.show()

### **(상위5%) 제품을 추가한 순서와 재주문 간의 관계**


In [ ]:
# orders와 user 데이터셋을 병합
merged_data = pd.merge(orders, user, on='order_id', how='inner')

In [ ]:
# 카트에 제품을 추가한 순서(add_to_cart_order)와 재주문 여부(reordered) 간의 관계 확인
cart_reorder_relation = merged_data.groupby('add_to_cart_order')['reordered'].mean()

In [ ]:
# cart_reorder_relation = 각 카트에 제품을 추가한 순서별로 평균 재주문 비율
cart_reorder_relation*100

In [ ]:
cart_reorder_relation = merged_data.groupby('add_to_cart_order')['reordered'].mean()

# Segment를 나누기 위한 구간 설정
segment_size = 10
max_order = merged_data['add_to_cart_order'].max()

# Segment 계산 및 새로운 컬럼 추가
merged_data['add_to_cart_segment'] = pd.cut(merged_data['add_to_cart_order'], bins=range(0, max_order + segment_size, segment_size), right=False)

# 재주문 비율 계산
cart_reorder_relation_segmented = merged_data.groupby('add_to_cart_segment')['reordered'].mean()

# 시각화 설정
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))

# 'add_to_cart_order'에 따른 재주문 비율 시각화
sns.barplot(x=cart_reorder_relation_segmented.index, y=(cart_reorder_relation_segmented*100).values, color='skyblue')

# 그래프 설정
plt.title('Reorder Rate by Add to Cart Order Segment')
plt.xlabel('Add to Cart Order Segment')
plt.ylabel('Reorder Rate')
plt.show()

In [ ]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 설정
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))

# 'add_to_cart_order'에 따른 재주문 비율 시각화
sns.barplot(x=cart_reorder_relation.index, y=(cart_reorder_relation*100).values, color='skyblue')

# 그래프 설정
plt.title('Reorder Rate by Add to Cart Order')
plt.xlabel('Add to Cart Order')
plt.ylabel('Reorder Rate')
plt.show()

## **(상위 5%) 특정 시간대별 재주문 분석**

In [ ]:
#주문 수와 함께 해당 시간대의 재주문 비율을 시각적으로 비교
plt.figure(figsize=(12, 6))
sns.countplot(x='order_hour_of_day',hue='reordered', data=merged_data)
plt.title('Reorder Count by Order Hour of Day')
plt.xlabel('Order Hour of Day')
plt.ylabel('Count')
plt.legend(title='Reordered', loc='upper right')
plt.show()

In [ ]:
orders_by_hour = merged_data.groupby('order_hour_of_day').size()
orders_by_hour

In [ ]:
plt.figure(figsize=(12, 6))
orders_by_hour.plot(kind='bar', color='skyblue')
plt.xlabel('order_hour_of_day')
plt.ylabel('count')
plt.show()

In [ ]:
# order_id 별 재주문률 (★지표★)
reordered_ratio = orders.groupby('order_id')['reordered'].mean().reset_index()
reordered_ratio*100

### **(상위5%)재구매 패턴 분석**
- "reordered" 컬럼을 사용하여 각 제품이 얼마나 자주 재주문되었는지 확인
- reorder_count 컬럼:해당 제품의 총 재주문 횟수

In [ ]:
 products=pd.read_csv(path+'products.csv')

In [ ]:
# 각 제품별로 재주문 횟수 계산
reorder_counts = orders.groupby('product_id')['reordered'].sum().reset_index()
reorder_counts.columns = ['product_id', 'reorder_count']
reorder_counts.sort_values(by='reorder_count',ascending=False)

In [ ]:
merged_product_name = pd.merge(reorder_counts , products, on='product_id', how='left')
merged_product_name.sort_values(by='reorder_count',ascending=False)
merged_product_name.loc[:,['product_id','reorder_count','product_name']].sort_values(by='reorder_count',ascending=False).head(30)

## **user loyalty 분석**
- 사용자의 첫 번째 주문과 그 이후 주문 간의 시간 간격 계산

In [ ]:
# 사용자별로 첫 번째 주문 이후 얼마나 오랜 기간 동안 주문을 유지하는지 계산
user_retention = user.groupby('user_id')['days_since_prior_order'].sum().reset_index()
user_retention.sort_values(by='days_since_prior_order',ascending=False).head(50)